<a href="https://colab.research.google.com/github/zunaed-farabe/MH-project/blob/main/Meta_Heuristic_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install deap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 1.5 MB/s eta 0:00:00


In [ ]:
import numpy as np
from scipy.sparse.linalg import svds
from deap import algorithms, base, creator, tools
global W
def learn_low_rank_representation(X, s):
    """
    Learn a low-rank representation of the dataset X.

    Parameters:
        X (numpy.ndarray): The dataset.
        s (int): The number of rows/columns to sample.

    Returns:
        W (numpy.ndarray): The low-rank representation.
        residual_error (numpy.ndarray): The residual error.
    """
    # Random sampling of s rows/columns
    indices = np.random.choice(X.shape[0], s, replace=False)
    C = X[indices, :]
    R = X[:, indices]

    # Compute SVD of concatenated matrices
    U, sigma, Vt = svds(np.hstack((C.T, R)), k=s)

    # Extract the low-rank representation
    W = np.dot(U[:, :s], np.diag(sigma[:s]))

    # Calculate the residual error
    reconstructed_X = np.dot(W, Vt[:s, :])


    X = X.reshape(-1)  # [1, 2, 3, 4, 5, 6]
    reconstructed_X = reconstructed_X.reshape(-1)
    max_size = max(len(X), len(reconstructed_X))
    X_padded = np.pad(X, (0, max_size - len(X)), mode='constant')
    reconstructed_X_padded = np.pad(reconstructed_X, (0, max_size - len(reconstructed_X)), mode='constant')
    residual_error = X_padded - reconstructed_X_padded


    return W, residual_error

def evolutionary_computation(W, residual_error, num_generations):
    """
    Perform evolutionary computation using a genetic algorithm.

    Parameters:
        W (numpy.ndarray): The low-rank representation.
        residual_error (numpy.ndarray): The residual error.
        num_generations (int): The number of generations.

    Returns:
        final_solution (numpy.ndarray): The final solution found by the genetic algorithm.
    """
    # Define the fitness function

    def evaluate(individual):
        global W
        individual = individual.reshape(-1)
        W = W.reshape(-1)
        max_size = max(len(W), len(individual))
        W_padded = np.pad(W, (0, max_size - len(W)), mode='constant')
        individual = np.pad(individual, (0, max_size - len(individual)), mode='constant')
        new_residual_error = W_padded.dot(individual)

        return np.sum(np.square(new_residual_error - residual_error))

    # Define genetic algorithm parameters
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", np.ndarray, fitness=creator.FitnessMin)

    toolbox = base.Toolbox()
    toolbox.register("attr_float", np.random.uniform, 0, 1)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=W.shape[1])
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("mate", tools.cxBlend, alpha=0.5)
    toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.1, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)
    toolbox.register("evaluate", evaluate)

    # Initialize population
    population = toolbox.population(n=100)

    # Perform evolution
    algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=num_generations, verbose=False)

    # Get the best individual
    final_solution = tools.selBest(population, k=1)[0]

    return final_solution

def main(X, s, num_generations):
    """
    Main function to combine the steps of learning low-rank representation
    and performing evolutionary computation.

    Parameters:
        X (numpy.ndarray): The dataset.
        s (int): The number of rows/columns to sample.
        num_generations (int): The number of generations for evolutionary computation.

    Returns:
        final_solution (numpy.ndarray): The final solution found by the evolutionary computation.
    """
    # Step 1: Learning a low-rank representation
    W, residual_error = learn_low_rank_representation(X, s)

    # Step 2: Performing evolutionary computation
    final_solution = evolutionary_computation(W, residual_error, num_generations)

    return final_solution

# Example usage:
X = np.random.rand(10, 10)
s = 4
num_generations = 5
result = main(X, s, num_generations)


# Loading data

In [50]:
def read_vrp_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    data = {}
    section = None
    for line in lines:
        line = line.strip()
        if not line:
            continue
        elif line.startswith('COMMENT'):
            pass
        elif line.startswith('NAME'):
            data['name'] = line.split(':')[1].strip()
        elif line.startswith('TYPE'):
            data['type'] = line.split(':')[1].strip()
        elif line.startswith('DIMENSION'):
            data['dimension'] = int(line.split(':')[1].strip())
        elif line.startswith('EDGE_WEIGHT_TYPE'):
            data['edge_weight_type'] = line.split(':')[1].strip()
        elif line.startswith('CAPACITY'):
            data['capacity'] = int(line.split(':')[1].strip())
        elif line.startswith('NODE_COORD_SECTION'):
            section = 'node_coords'
            data['node_coords'] = []
        elif line.startswith('DEMAND_SECTION'):
            section = 'demands'
            data['demands'] = []
        elif line.startswith('DEPOT_SECTION'):
            section = 'depots'
            data['depots'] = []
        elif section == 'node_coords':
            node_id, x, y = map(float, line.split())
            data['node_coords'].append((node_id, x, y))
        elif section == 'demands':
            node_id, demand = map(int, line.split())
            data['demands'].append((node_id, demand))
        elif section == 'depots':
            if line.startswith('EOF'):
              break
            else:
              node_id = int(line.split()[0])
              data['depots'].append(node_id)

    return data

file_path = '/content/gil262.vrp'
vrp_data = read_vrp_file(file_path)

# print(vrp_data)
# display(vrp_data["node_coords"][10:30])

In [19]:
type(vrp_data["node_coords"])

list

In [51]:
coordinates = []
for cord in vrp_data["node_coords"]:
  coordinates.append([cord[1], cord[1]])

array_coord = np.array(coordinates)
# Find the minimum value
min_val = np.min(array_coord)
print("Minimum value:", min_val)

# Find the maximum value
max_val = np.max(array_coord)
print("Maximum value:", max_val)

Minimum value: -99.0
Maximum value: 99.0


In [52]:
# Get the size of the original array
size = array_coord.shape[0]

# Create a square matrix of the same size filled with zeros
square_matrix = np.zeros((size, size))

# Assign the original values to their corresponding positions in the square matrix
for i, (x, y) in enumerate(array_coord):
    square_matrix[i, 0] = x
    square_matrix[i, 1] = y


print(square_matrix)

[[-33. -33.   0. ...   0.   0.   0.]
 [-99. -99.   0. ...   0.   0.   0.]
 [-59. -59.   0. ...   0.   0.   0.]
 ...
 [ 40.  40.   0. ...   0.   0.   0.]
 [-60. -60.   0. ...   0.   0.   0.]
 [-60. -60.   0. ...   0.   0.   0.]]


## Low Rank + (1+1) Evolutionary Strategy

In [53]:

from scipy.sparse.linalg import svds

def learn_low_rank_representation(X, s):
    """
    Learn a low-rank representation of the dataset X.

    Parameters:
        X (numpy.ndarray): The dataset.
        s (int): The number of rows/columns to sample.

    Returns:
        W (numpy.ndarray): The low-rank representation.
        residual_error (numpy.ndarray): The residual error.
    """
    # Random sampling of s rows/columns
    indices = np.random.choice(X.shape[0], s, replace=False)
    C = X[indices, :]
    R = X[:, indices]

    # Compute SVD of concatenated matrices
    U, sigma, Vt = svds(np.hstack((C.T, R)), k=s)

    # Extract the low-rank representation
    W = np.dot(U[:, :s], np.diag(sigma[:s]))

    # Calculate the residual error
    reconstructed_X = np.dot(W, Vt[:s, :])


    X = X.reshape(-1)  # [1, 2, 3, 4, 5, 6]
    reconstructed_X = reconstructed_X.reshape(-1)
    max_size = max(len(X), len(reconstructed_X))
    X_padded = np.pad(X, (0, max_size - len(X)), mode='constant')
    reconstructed_X_padded = np.pad(reconstructed_X, (0, max_size - len(reconstructed_X)), mode='constant')
    residual_error = X_padded - reconstructed_X_padded


    return W, residual_error

# Define the fitness function
def evaluate(individual,W,residual_error):


    individual = individual.reshape(-1)
    W = W.reshape(-1)
    max_size = max(len(W), len(individual))
    W_padded = np.pad(W, (0, max_size - len(W)), mode='constant')
    individual = np.pad(individual, (0, max_size - len(individual)), mode='constant')
    new_residual_error = W_padded.dot(individual)
    return np.sum(np.square(new_residual_error - residual_error))

def evolutionary_computation(W, residual_error, num_generations):
    """
    Perform evolutionary computation using the (1+1)-Evolution Strategy.

    Parameters:
        W (numpy.ndarray): The low-rank representation.
        residual_error (numpy.ndarray): The residual error.
        num_generations (int): The number of generations.

    Returns:
        final_solution (numpy.ndarray): The final solution found by the evolutionary strategy.
    """
    # Initialize the parent solution
    parent = np.random.uniform(0, 1, W.shape[1])

    # Perform (1+1)-ES
    for _ in range(num_generations):
        # Create an offspring by mutating the parent
        offspring = parent + np.random.normal(0, 0.1, W.shape[1])

        # Evaluate fitness of parent and offspring
        parent_fitness = evaluate(parent,W,residual_error)
        offspring_fitness = evaluate(offspring,W,residual_error)

        # Select the better solution for the next generation
        if offspring_fitness < parent_fitness:
            parent = offspring

    # The final solution is the parent after the last generation
    final_solution = parent

    return final_solution

def main(X, s, num_generations):
    """
    Main function to combine the steps of learning low-rank representation
    and performing evolutionary computation.

    Parameters:
        X (numpy.ndarray): The dataset.
        s (int): The number of rows/columns to sample.
        num_generations (int): The number of generations for evolutionary computation.

    Returns:
        final_solution (numpy.ndarray): The final solution found by the evolutionary computation.
    """
    # Step 1: Learning a low-rank representation
    W, residual_error = learn_low_rank_representation(X, s)

    # Step 2: Performing evolutionary computation
    final_solution = evolutionary_computation(W, residual_error, num_generations)

    return final_solution

# Example usage:
# X = np.random.rand(100, 100)
X = square_matrix
s = 2
num_generations = 2
result = main(X, s, num_generations)
print(result)

[0.93085215 0.6578769 ]


In [47]:
import numpy as np

def low_rank_representation(data, sample_size):
    """
    Performs low-rank representation on the given data.

    Args:
        data: A 2D numpy array representing the data points.
        sample_size: The number of samples to use for reconstruction.

    Returns:
        A reconstructed 2D numpy array representing the low-rank approximation.
    """
    M, N = data.shape  # Get dimensions of the data

    # Sample data points (replace with your sampling strategy)
    indices = np.random.choice(M, sample_size, replace=False)
    sampled_data = data[indices, :]

    # Perform Singular Value Decomposition (SVD)
    U, S, Vt = np.linalg.svd(sampled_data)

    # Choose the top singular values and vectors
    Ur = U[:, :sample_size]
    Sr = np.diag(S[:sample_size])
    Vtr = Vt[:sample_size, :]

    # Reconstruct data using low-rank representation
    reconstructed_data = np.dot(Ur, np.dot(Sr, Vtr))

    return reconstructed_data

# Example usage
data = np.random.rand(100, 100)  # Replace with your actual data
sample_size = 10  # Adjust sample size as needed

reconstructed_data = low_rank_representation(data, sample_size)

print(reconstructed_data.shape)  # Should be the same as data.shape


(10, 100)
